In [1]:
import os
from dotenv import load_dotenv
from pathlib import Path

env_path = Path('..') / '.env.local'
load_dotenv(dotenv_path=env_path)

url = os.getenv("NEXT_PUBLIC_SUPABASE_URL")
key = os.getenv("NEXT_PUBLIC_SUPABASE_ANON_KEY")

In [2]:
from supabase import create_client, Client

In [3]:
supabase: Client = create_client(url, key)


In [115]:
def generate_autodraft_rankings(pool_id, draft_num):
    competition_id = supabase.table('pool').select('competition_id').eq('pool_id', pool_id).execute().data[0]['competition_id']
    print(competition_id)
    round_num = supabase.table('poolrule_draft').select('round_num').eq('pool_id', pool_id).eq('draft_num', draft_num).execute().data[0]['round_num']
    print(round_num)
    ranked_players = supabase.table('player_total_score_view').select('player_unique, total_points').eq('competition_id', competition_id).order('total_points', desc=True, nullsfirst=False).execute().data
    # print(ranked_players)
    available_players = [i['player_unique'] for i in supabase.table('view_available_players').select('player_unique').eq('pool_id', pool_id).execute().data]
    # rankings = [f"{i['player_unique']}-{i['total_points']}" for i in ranked_players if i['player_unique'] in available_players]
    rankings = [i['player_unique'] for i in ranked_players if i['player_unique'] in available_players]
    return rankings

In [80]:
generate_autodraft_rankings(8, 2)

1
6
['bryan-greenlee-1', 'micah-parrish-1', 'andre-jackson-8', 'vladislav-goldin-1', 'nahiem-alleyne-1', 'nicholas-boyd-1', 'keshad-johnson-1', 'giancarlo-rosado-1', 'joey-calcaterra-1', 'nathan-mensah-1', 'brandon-weatherspoon-1', 'aguek-arop-1', 'michael-forrest-2', 'bensley-joseph-1', 'anthony-walker-7', 'adam-seiko-1', 'jalen-gaffney-1', 'harlond-beverly-1', 'jared-barnett-1', 'richie-springs-1', 'apostolos-roumoglou-1', 'triston-broughton-1', 'alejandro-ralat-1', 'brenen-lorient-1', 'cade-alger-1', 'yarin-hasson-1', 'isaiah-gaines-1', 'andrew-hurley-1', 'tre-carroll-1', 'favour-aire-1', 'elijah-saunders-1', 'miles-byrd-1', 'emmett-hendry-1', 'filippos-gkogkos-1', 'aj-casey-1', 'danilo-jovanovich-1', 'hassan-diarra-1', 'andre-johnson-jr-1', 'jakai-robinson-1', 'demarshay-johnsonjr-1', 'tyler-broughton-1', 'christian-watson-1', 'samson-johnson-1']


In [130]:
def generate_rankings_dict(pool_id, draft_num: int = 1, max_players: int = 1000, num_participants: int = 9):
    rostrankmap = {}
    draft_order = get_draft_order(draft_num=draft_num, pool_id=pool_id, num_participants=num_participants)
    rankings = supabase.table('draft_view').select('ranking, roster_id, player_unique').eq('pool_id', pool_id).eq('draft_num', draft_num).execute()
    print(len(rankings.data))
    for row in rankings.data:
        if row['roster_id'] == 11:
            print(row)

        if 'roster_id' not in row:
            continue
        roster_id = str(row['roster_id'])
        ranking = int(row['ranking'])
        player_unique = row['player_unique']
        if roster_id not in rostrankmap:
            rostrankmap[roster_id] = [None for i in range(0, max_players)]
            rostrankmap[roster_id][ranking] = player_unique
        else:
            rostrankmap[roster_id][ranking] = player_unique
    cleaned_map = {}
    for rost in rostrankmap:
        mapped_rankings = rostrankmap[rost]
        cleaned_map[str(rost)] = [i for i in mapped_rankings if i is not None]
    for rost in draft_order:
        if rost not in cleaned_map:
            cleaned_map[str(rost)] = generate_autodraft_rankings(pool_id, draft_num)
    return cleaned_map

In [85]:
print(generate_rankings_dict(8, 2))

1980
{'ranking': 121, 'roster_id': 11, 'player_unique': 'ryan-nembhard-1'}
{'ranking': 572, 'roster_id': 11, 'player_unique': 'joseph-pinion-1'}
{'ranking': 573, 'roster_id': 11, 'player_unique': 'kenneth-nwuba-1'}
{'ranking': 574, 'roster_id': 11, 'player_unique': 'john-hugley-1'}
{'ranking': 575, 'roster_id': 11, 'player_unique': 'francisco-caffaro-1'}
{'ranking': 576, 'roster_id': 11, 'player_unique': 'tre-donaldson-1'}
{'ranking': 577, 'roster_id': 11, 'player_unique': 'zach-clemence-1'}
{'ranking': 578, 'roster_id': 11, 'player_unique': 'demarion-watson-1'}
{'ranking': 579, 'roster_id': 11, 'player_unique': 'cj-gunn-1'}
{'ranking': 580, 'roster_id': 11, 'player_unique': 'cameron-tweedy-1'}
{'ranking': 582, 'roster_id': 11, 'player_unique': 'daimion-collins-1'}
{'ranking': 583, 'roster_id': 11, 'player_unique': 'adou-thiero-1'}
{'ranking': 584, 'roster_id': 11, 'player_unique': 'ryan-elvin-1'}
{'ranking': 585, 'roster_id': 11, 'player_unique': 'christian-reeves-1'}
{'ranking': 586,

TypeError: 'int' object is not iterable

In [51]:
def get_active_players(current_round: int = 2, competition_id: int = 1):
    available_players = supabase.table('view_active_players').select('player_unique, round_eliminated').eq('competition_id', competition_id).execute()
    return set([row['player_unique'] for row in available_players.data if row['round_eliminated'] == None])

In [6]:
print(get_active_players(current_round=6, competition_id=1))

{'jakai-robinson-1', 'wooga-poplar-1', 'yarin-hasson-1', 'andre-johnson-jr-1', 'brandon-weatherspoon-1', 'filippos-gkogkos-1', 'demarshay-johnsonjr-1', 'tristen-newton-1', 'anthony-walker-7', 'isaiah-gaines-1', 'isaiah-wong-1', 'nijel-pack-1', 'lamont-butler-1', 'vladislav-goldin-1', 'andrew-hurley-1', 'bryan-greenlee-1', 'micah-parrish-1', 'alejandro-ralat-1', 'nicholas-boyd-1', 'jaedon-ledee-1', 'harlond-beverly-1', 'nathan-mensah-1', 'danilo-jovanovich-1', 'cade-alger-1', 'favour-aire-1', 'darrion-trammell-1', 'tyler-broughton-1', 'matt-bradley-1', 'jordan-miller-2', 'hassan-diarra-1', 'donovan-clingan-1', 'giancarlo-rosado-1', 'triston-broughton-1', 'alijah-martin-1', 'jordan-hawkins-1', 'adam-seiko-1', 'jared-barnett-1', 'jalen-gaffney-1', 'richie-springs-1', 'apostolos-roumoglou-1', 'johnell-davis-1', 'brenen-lorient-1', 'emmett-hendry-1', 'alex-karaban-1', 'nahiem-alleyne-1', 'aj-casey-1', 'miles-byrd-1', 'aguek-arop-1', 'adama-sanogo-1', 'tre-carroll-1', 'christian-watson-1', '

# Draft

In [141]:
def run_draft(expected_players: int = 9, round_start: int = 2, round_end: int = 7, pool_id: int = 8, draft_num: int = 1):
    draft_rules = supabase.table('poolrule_draft').select('*').eq('pool_id', pool_id).eq('draft_num', draft_num).execute()
    print(draft_rules)
    roster_count = draft_rules.data[0]['roster_count']
    round_num = draft_rules.data[0]['round_num']
    draft_time = draft_rules.data[0]['draft_time']
    draft_order = draft_rules.data[0]['draft_order']
    draft_order = get_draft_order(num_participants=expected_players, pool_id=pool_id, draft_num=draft_num)
    rankings_dict = generate_rankings_dict(pool_id, draft_num)
    available_players_set = get_active_players()
    last_pick_dict = {str(key): 0 for key in draft_order}
    print(last_pick_dict)
    print(draft_order)
    pick_num = 163
    draft_insert = []
    # print(rankings_dict)
    print(rankings_dict.keys())
    print(":::RANKINGS DICT:::", rankings_dict)
    print(available_players_set)
    print(last_pick_dict)
    print(draft_order)
    print("Starting Draft")
    for draft_round in range (0, roster_count):
        for round_pick in range (0, expected_players):
            made_pick = False
            current_drafter = str(draft_order[round_pick])
            for pick in range (last_pick_dict[current_drafter], len(rankings_dict[current_drafter])):
                player_unique = rankings_dict[current_drafter][pick]
                if player_unique in available_players_set:
                    print(f'Player {player_unique} picked by {current_drafter} at pick {pick_num}')
                    last_pick_dict[current_drafter] = pick
                    available_players_set.remove(player_unique)
                    pick_row = {
                        'roster_id': int(current_drafter),
                        'player_unique': player_unique,
                        'round_start': round_start,
                        'round_end': round_end,
                        'pick_number': pick_num,
                        'draft_num': draft_num
                    }
                    draft_insert.append(pick_row)
                    made_pick = True
                    break
            if not made_pick:
                raise Exception(f'No pick made by {current_drafter} at pick {pick_num} round {draft_round+1}')
            pick_num += 1
            if round_pick == expected_players - 1:
                print(f'Round {draft_round} complete')
                draft_order.reverse()
    print(draft_insert)
    supabase.table('roster_player').upsert( draft_insert, ignore_duplicates=True ).execute()

In [142]:
run_draft(round_start=6, draft_num=2, expected_players=9)

data=[{'draft_num': 2, 'pool_id': 8, 'roster_count': 2, 'draft_time': '2023-04-01T12:00:00', 'draft_order': -1, 'round_num': 6}] count=None
126
1
6
1
6
1
6
1
6
1
6
1
6
1
6
1
6
1
6
{'11': 0, '9': 0, '13': 0, '14': 0, '16': 0, '12': 0, '17': 0, '10': 0, '18': 0}
[11, 9, 13, 14, 16, 12, 17, 10, 18]
dict_keys(['10', '14', '16', '17', '11', '9', '13', '12', '18'])
:::RANKINGS DICT::: {'10': ['bryan-greenlee-1', 'micah-parrish-1', 'andre-jackson-8', 'vladislav-goldin-1', 'nahiem-alleyne-1', 'nicholas-boyd-1', 'keshad-johnson-1', 'giancarlo-rosado-1', 'joey-calcaterra-1', 'nathan-mensah-1', 'brandon-weatherspoon-1', 'aguek-arop-1', 'michael-forrest-2', 'bensley-joseph-1', 'anthony-walker-7', 'adam-seiko-1', 'jalen-gaffney-1', 'harlond-beverly-1', 'jared-barnett-1', 'richie-springs-1', 'apostolos-roumoglou-1', 'triston-broughton-1', 'alejandro-ralat-1', 'brenen-lorient-1', 'cade-alger-1', 'yarin-hasson-1', 'isaiah-gaines-1', 'andrew-hurley-1', 'tre-carroll-1', 'favour-aire-1', 'elijah-saunders

In [124]:
import random
import datetime
import json
def generate_draft_order(pool_id: int = 8, draft_num: int = 1):
    rosters = supabase.table('roster_full_view').select('roster_id, rankings_submitted').eq('pool_id', pool_id).execute()
    draft_rules = supabase.table('poolrule_draft').select('*').eq('pool_id', pool_id).eq('draft_num', draft_num).execute()
    draft_rule_order = draft_rules.data[0]['draft_order']
    roster_list = []
    if draft_num == 1 or draft_rules.data[0]['draft_order'] == 0:
        roster_list = [row['roster_id'] for row in rosters.data]
        random.shuffle(roster_list)
    else:
        prev_draft_order = [row['roster_id'] for row in supabase.table('roster_draftorder').select('roster_id').eq('draft_num', draft_num-1).execute().data]
        if draft_rule_order == 1:
            roster_list = prev_draft_order
        elif draft_rule_order == -1:
            prev_draft_order.reverse()
            roster_list = prev_draft_order
        print(roster_list)
    order_insert = []
    created_at = json.dumps(datetime.datetime.now(), indent=4, sort_keys=True, default=str)
    for idx, roster_id in enumerate(roster_list):
        row = {
            'draft_num': draft_num,
            'roster_id': roster_id,
            'created_at': created_at,
            'draft_order': idx+1
        }
        order_insert.append(row)
    supabase.table('roster_draftorder').upsert( order_insert, on_conflict='draft_num, roster_id', ignore_duplicates=False).execute()
    return roster_list

In [125]:
generate_draft_order(draft_num=2)

[11, 9, 13, 14, 16, 12, 17, 10, 18]

In [89]:
def get_draft_order(num_participants: int = 9, pool_id=8, draft_num=1):
    draft_order_data = supabase.table('draft_order_view').select('*').eq('pool_id', pool_id).eq('draft_num', draft_num).execute()
    if len(draft_order_data.data) == num_participants:
        draft_order = [None for i in range(0, num_participants + 1)]
        for row in draft_order_data.data:
            draft_order[row['draft_order']] = row['roster_id']
    else:
        draft_order = generate_draft_order(pool_id, draft_num)
    return [i for i in draft_order if i is not None]

In [135]:
print(get_draft_order())

[18, 10, 17, 12, 16, 14, 13, 9, 11]


In [83]:
get_draft_info()

[{'draft_num': 1, 'pool_id': 8, 'roster_count': 12, 'draft_time': '2023-03-16T11:00:00', 'draft_order': 0, 'round_num': 2}]


In [74]:
generate_rankings_dict(pool_id=8)

data=[{'draft_num': 1, 'pool_id': 8, 'roster_count': 12, 'draft_time': '2023-03-16T11:00:00', 'draft_order': 0, 'round_num': 2}] count=None
{'ranking': 48, 'roster_id': 12, 'player_unique': 'jayden-gardner-1'}
{'ranking': 48, 'roster_id': 12, 'player_unique': 'jayden-gardner-1'}
{'ranking': 49, 'roster_id': 12, 'player_unique': 'mitchell-saxen-1'}
{'ranking': 49, 'roster_id': 12, 'player_unique': 'mitchell-saxen-1'}
{'ranking': 50, 'roster_id': 12, 'player_unique': 'anton-watson-1'}
{'ranking': 50, 'roster_id': 12, 'player_unique': 'anton-watson-1'}
{'ranking': 51, 'roster_id': 12, 'player_unique': 'fletcher-loyer-1'}
{'ranking': 51, 'roster_id': 12, 'player_unique': 'fletcher-loyer-1'}
{'ranking': 52, 'roster_id': 12, 'player_unique': 'kj-adamsjr-1'}
{'ranking': 52, 'roster_id': 12, 'player_unique': 'kj-adamsjr-1'}
{'ranking': 53, 'roster_id': 12, 'player_unique': 'jarace-walker-1'}
{'ranking': 53, 'roster_id': 12, 'player_unique': 'jarace-walker-1'}
{'ranking': 54, 'roster_id': 12, '

In [ ]:
def update_tables():

In [ ]:
def run_draft(round_start, round_end):
